In [1]:
import pandas as pd
import re
import os
from google.cloud import storage
import numpy as np

# Nếu bạn có service account key file, uncomment dòng dưới:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/service-account-key.json'

df = pd.read_csv("batdongsan.csv")


In [2]:
df.head(5)


,Diện tích sử dụng,Mã BĐS,price,Pháp lý,Ngày đăng,Nhà tắm,Phòng ngủ,address,Diện tích đất
0,100 m2,22712100,5 tỷ 800 triệu,Sổ đỏ,15/10/2025,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN
1,80 m2,22712094,4 tỷ 350 triệu,Sổ đỏ,15/10/2025,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
2,90 m2,22711721,4 tỷ 580 triệu,Sổ đỏ,15/10/2025,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN
3,120 m2,22711722,4 tỷ 700 triệu,Sổ đỏ,15/10/2025,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
4,100 m2,22711723,5 tỷ 400 triệu,Sổ đỏ,15/10/2025,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN


In [3]:
df = df.rename(columns={
    'Diện tích sử dụng' : 'dien_tich_su_dung_m2',
    'Mã BĐS' : 'ma_bds',
    'price' : 'gia_ty',
    'Pháp lý' : 'phap_ly',
    'Ngày đăng' : 'ngay_dang',
    'Nhà tắm'  : 'nha_tam',
    'Phòng ngủ': 'phong_ngu',
    'address': 'dia_chi',
    'Diện tích đất': 'dien_tich_dat_m2'
})

In [4]:
df.head(5)

,dien_tich_su_dung_m2,ma_bds,gia_ty,phap_ly,ngay_dang,nha_tam,phong_ngu,dia_chi,dien_tich_dat_m2
0,100 m2,22712100,5 tỷ 800 triệu,Sổ đỏ,15/10/2025,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN
1,80 m2,22712094,4 tỷ 350 triệu,Sổ đỏ,15/10/2025,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
2,90 m2,22711721,4 tỷ 580 triệu,Sổ đỏ,15/10/2025,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN
3,120 m2,22711722,4 tỷ 700 triệu,Sổ đỏ,15/10/2025,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
4,100 m2,22711723,5 tỷ 400 triệu,Sổ đỏ,15/10/2025,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN


In [5]:
# dien  tich su dung
df['dien_tich_su_dung_m2'] = df['dien_tich_su_dung_m2'].str.extract(r'(\d+(?:\.\d+)?)').astype(float)



In [ ]:


# Làm sạch dữ liệu - chuyển string thành NaN
def clean_room_data(value):
    if pd.isna(value):
        return None
    if isinstance(value, str):
        # Nếu là string như 'Nhà tắm', 'Phòng ngủ' thì chuyển thành NaN
        if value.lower() in ['nhà tắm', 'phòng ngủ', 'phong ngu', 'nha tam']:
            return None
        # Nếu là số dạng string, chuyển thành số
        try:
            return float(value)
        except:
            return None
    return value

# Làm sạch dữ liệu
df['nha_tam'] = df['nha_tam'].apply(clean_room_data)
df['phong_ngu'] = df['phong_ngu'].apply(clean_room_data)


# Phân tích mối quan hệ giữa diện tích và số phòng
# Tính trung bình số phòng theo diện tích
df_with_rooms = df[(df['nha_tam'].notna()) & (df['phong_ngu'].notna())].copy()
if len(df_with_rooms) > 0:
    # Tạo bins cho diện tích
    df_with_rooms['dien_tich_bin'] = pd.cut(df_with_rooms['dien_tich_su_dung_m2'], 
                                           bins=[0, 50, 80, 120, 200, float('inf')], 
                                           labels=['<50m²', '50-80m²', '80-120m²', '120-200m²', '>200m²'])
    
    # Tính trung bình số phòng theo từng bin diện tích
    avg_rooms_by_area = df_with_rooms.groupby('dien_tich_bin', observed=True).agg({
        'nha_tam': 'mean',
        'phong_ngu': 'mean'
    }).round(0)
    
    
    # Hàm fill dựa trên diện tích
    def fill_rooms_by_area(row):
        if pd.isna(row['nha_tam']) or pd.isna(row['phong_ngu']):
            area = row['dien_tich_su_dung_m2']
            if pd.isna(area):
                return row['nha_tam'], row['phong_ngu']
            
            # Xác định bin diện tích
            if area < 50:
                bin_name = '<50m²'
            elif area < 80:
                bin_name = '50-80m²'
            elif area < 120:
                bin_name = '80-120m²'
            elif area < 200:
                bin_name = '120-200m²'
            else:
                bin_name = '>200m²'
            
            # Fill từ trung bình của bin tương ứng
            if bin_name in avg_rooms_by_area.index:
                nha_tam_fill = avg_rooms_by_area.loc[bin_name, 'nha_tam'] if pd.isna(row['nha_tam']) else row['nha_tam']
                phong_ngu_fill = avg_rooms_by_area.loc[bin_name, 'phong_ngu'] if pd.isna(row['phong_ngu']) else row['phong_ngu']
                return nha_tam_fill, phong_ngu_fill
        
        return row['nha_tam'], row['phong_ngu']
    
    # Apply hàm fill
    df[['nha_tam', 'phong_ngu']] = df.apply(fill_rooms_by_area, axis=1, result_type='expand')
    
    # Fill các giá trị còn lại bằng trung bình chung
    if df['nha_tam'].isna().sum() > 0:
        df['nha_tam'] = df['nha_tam'].fillna(df['nha_tam'].mean())
    if df['phong_ngu'].isna().sum() > 0:
        df['phong_ngu'] = df['phong_ngu'].fillna(df['phong_ngu'].mean())
    
    # Convert về int - làm tròn trước khi convert
    df['nha_tam'] = df['nha_tam'].round().astype('Int64')
    df['phong_ngu'] = df['phong_ngu'].round().astype('Int64')
    

C:\Users\Admin\AppData\Local\Temp\ipykernel_28056\191046072.py:69: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nha_tam'].fillna(df['nha_tam'].mean(), inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_28056\191046072.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [7]:
df = df.drop_duplicates(subset=['ma_bds']).reset_index(drop=True)
df['ma_bds'] = df['ma_bds'].astype(str)
df['nha_tam'] = pd.to_numeric(df['nha_tam'], errors='coerce').astype('Int64')
df['phong_ngu'] = pd.to_numeric(df['phong_ngu'], errors='coerce').astype('Int64')

In [8]:
def chuan_hoa_gia(x):
    x = str(x).lower()
    ty = 0
    trieu = 0

    # Lấy phần 'tỷ'
    match_ty = re.search(r'(\d+(?:[\.,]\d+)?)\s*t[ỷi]', x)
    if match_ty:
        ty = float(match_ty.group(1).replace(',', '.'))

    # Lấy phần 'triệu'
    match_trieu = re.search(r'(\d+(?:[\.,]\d+)?)\s*triệu', x)
    if match_trieu:
        trieu = float(match_trieu.group(1).replace(',', '.'))

    # Đổi về tỷ đồng
    return round(ty + trieu/1000, 3)

def tach_dia_chi(dia_chi):
    dia_chi = str(dia_chi)
    
    # Tên đường (trước dấu phẩy đầu tiên)
    ten_duong = dia_chi.split(",")[0].strip()

    # Tên phường/xã
    phuong = re.search(r'Phường\s+([^,]+)', dia_chi)
    phuong = phuong.group(1).strip() if phuong else None

    # Tên quận/huyện
    quan = re.search(r'Quận\s+([^,]+)', dia_chi)
    quan = quan.group(1).strip() if quan else None

    # Thành phố / Tỉnh
    tp = re.search(r'(Thành phố|TP\.?|Tỉnh)\s+([^,]+)', dia_chi)
    tp = tp.group(2).strip() if tp else "Hà Nội"

    # Gộp tên đường + phường
    phuong_xa = f"{ten_duong} - Phường {phuong}" if phuong else ten_duong

    return pd.Series([phuong_xa, quan, tp])


# Hàm làm sạch diện tích
def clean_dien_tich(value):
    if pd.isna(value):
        return None
    value = str(value).lower().strip()
    
    # Tìm số đầu tiên trong chuỗi (có thể có thập phân)
    match = re.search(r'(\d+(\.\d+)?)', value)
    if match:
        return float(match.group(1))
    else:
        return None



df['dien_tich_dat_m2'] = df['dien_tich_dat_m2'].apply(clean_dien_tich)
df['gia(ty)'] = df['gia_ty'].apply(chuan_hoa_gia)
df[['phuong_xa', 'quan_huyen', 'thanh_pho']] = df['dia_chi'].apply(tach_dia_chi)




In [9]:
for c in ['phap_ly','phuong_xa','quan_huyen','thanh_pho']:
    df[c] = df[c].astype(str).str.strip()

In [10]:
df['ngay_dang'] = pd.to_datetime(df['ngay_dang'], format='%d/%m/%Y', errors='coerce')

In [11]:
df.head(5)

,dien_tich_su_dung_m2,ma_bds,gia_ty,phap_ly,ngay_dang,nha_tam,phong_ngu,dia_chi,dien_tich_dat_m2,gia(ty),phuong_xa,quan_huyen,thanh_pho
0,100.0,22712100,5 tỷ 800 triệu,Sổ đỏ,2025-10-15,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN,5.80,Đại La - Phường Đồng Nhân,Hai Bà Trưng,Hà Nội
1,80.0,22712094,4 tỷ 350 triệu,Sổ đỏ,2025-10-15,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN,4.35,Thanh Nhàn - Phường Thanh Nhàn,Hai Bà Trưng,Hà Nội
2,90.0,22711721,4 tỷ 580 triệu,Sổ đỏ,2025-10-15,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN,4.58,Nhân Hòa - Phường Nhân Chính,Thanh Xuân,Hà Nội
3,120.0,22711722,4 tỷ 700 triệu,Sổ đỏ,2025-10-15,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN,4.70,Thanh Nhàn - Phường Thanh Nhàn,Hai Bà Trưng,Hà Nội
4,100.0,22711723,5 tỷ 400 triệu,Sổ đỏ,2025-10-15,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN,5.40,Yên Lạc - Phường Vĩnh Tuy,Hai Bà Trưng,Hà Nội


In [12]:

df_tmp = df[['ma_bds','dien_tich_su_dung_m2','gia(ty)','phap_ly','ngay_dang','phuong_xa','quan_huyen','thanh_pho','nha_tam','phong_ngu','dien_tich_dat_m2']]


df_tmp = df_tmp[(df_tmp['gia(ty)'] != 0.0) & (df_tmp['gia(ty)'].notna())]


In [ ]:
# KIỂM TRA BUSINESS LOGIC VALIDATION
print("=== BUSINESS LOGIC VALIDATION ===")

# 1. Kiểm tra giá hợp lý
print("1. GIÁ HỢP LÝ:")
print(f"   - Giá min: {df_tmp['gia(ty)'].min():.3f} tỷ")
print(f"   - Giá max: {df_tmp['gia(ty)'].max():.3f} tỷ")
print(f"   - Giá trung bình: {df_tmp['gia(ty)'].mean():.3f} tỷ")

# 2. Kiểm tra diện tích hợp lý
print("\n2. DIỆN TÍCH HỢP LÝ:")
print(f"   - Diện tích min: {df_tmp['dien_tich_su_dung_m2'].min():.1f} m²")
print(f"   - Diện tích max: {df_tmp['dien_tich_su_dung_m2'].max():.1f} m²")
print(f"   - Diện tích trung bình: {df_tmp['dien_tich_su_dung_m2'].mean():.1f} m²")

# 3. Tính giá/m²
df_tmp['gia_per_m2'] = (df_tmp['gia(ty)'] * 1000) / df_tmp['dien_tich_su_dung_m2']
print(f"\n3. GIÁ/M²:")
print(f"   - Giá/m² min: {df_tmp['gia_per_m2'].min():.0f} triệu VND/m²")
print(f"   - Giá/m² max: {df_tmp['gia_per_m2'].max():.0f} triệu VND/m²")
print(f"   - Giá/m² trung bình: {df_tmp['gia_per_m2'].mean():.0f} triệu VND/m²")

# 4. PHÁT HIỆN VÀ XỬ LÝ OUTLIERS
print("\n4. PHÁT HIỆN OUTLIERS:")

# Phát hiện outliers bằng IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Phát hiện outliers cho giá
price_outliers, price_lower, price_upper = detect_outliers_iqr(df_tmp, 'gia(ty)')
print(f"   - Giá outliers: {len(price_outliers)} records")
print(f"   - Giá range hợp lý: {price_lower:.3f} - {price_upper:.3f} tỷ")

# Phát hiện outliers cho diện tích
area_outliers, area_lower, area_upper = detect_outliers_iqr(df_tmp, 'dien_tich_su_dung_m2')
print(f"   - Diện tích outliers: {len(area_outliers)} records")
print(f"   - Diện tích range hợp lý: {area_lower:.1f} - {area_upper:.1f} m²")

# Phát hiện outliers cho giá/m²
price_per_m2_outliers, price_per_m2_lower, price_per_m2_upper = detect_outliers_iqr(df_tmp, 'gia_per_m2')
print(f"   - Giá/m² outliers: {len(price_per_m2_outliers)} records")
print(f"   - Giá/m² range hợp lý: {price_per_m2_lower:.0f} - {price_per_m2_upper:.0f} triệu VND/m²")

# 5. LỌC OUTLIERS NGHIÊM TRỌNG
print("\n5. LỌC OUTLIERS NGHIÊM TRỌNG:")

# Định nghĩa ngưỡng hợp lý cho bất động sản Hà Nội
REASONABLE_PRICE_MAX = 50  # 50 tỷ
REASONABLE_AREA_MAX = 500  # 500 m²
REASONABLE_PRICE_PER_M2_MAX = 200  # 200 triệu VND/m²

# Lọc dữ liệu hợp lý
df_clean = df_tmp[
    (df_tmp['gia(ty)'] <= REASONABLE_PRICE_MAX) &
    (df_tmp['dien_tich_su_dung_m2'] <= REASONABLE_AREA_MAX) &
    (df_tmp['gia_per_m2'] <= REASONABLE_PRICE_PER_M2_MAX)
].copy()

print(f"   - Dữ liệu gốc: {len(df_tmp):,} records")
print(f"   - Dữ liệu sau lọc: {len(df_clean):,} records")
print(f"   - Đã loại bỏ: {len(df_tmp) - len(df_clean):,} records ({((len(df_tmp) - len(df_clean))/len(df_tmp)*100):.1f}%)")

# Cập nhật df_tmp với dữ liệu đã lọc
df_tmp = df_clean.copy()


=== BUSINESS LOGIC VALIDATION ===
1. GIÁ HỢP LÝ:
   - Giá min: 0.002 tỷ
   - Giá max: 86900.000 tỷ
   - Giá trung bình: 51.169 tỷ

2. DIỆN TÍCH HỢP LÝ:
   - Diện tích min: 1.0 m²
   - Diện tích max: 6300.0 m²
   - Diện tích trung bình: 126.7 m²

3. GIÁ/M²:
   - Giá/m² min: 0 triệu VND/m²
   - Giá/m² max: 790000 triệu VND/m²
   - Giá/m² trung bình: 714 triệu VND/m²


In [ ]:
# ===== KIỂM TRA CHẤT LƯỢNG DỮ LIỆU SAU KHI LỌC =====

print("=== CHẤT LƯỢNG DỮ LIỆU SAU KHI LỌC OUTLIERS ===")

# 1. Thống kê mô tả sau khi lọc
print("1. THỐNG KÊ MÔ TẢ:")
print(f"   - Số records: {len(df_tmp):,}")
print(f"   - Số cột: {len(df_tmp.columns)}")

# 2. Kiểm tra missing values
print("\n2. MISSING VALUES:")
missing_data = df_tmp.isnull().sum()
missing_percent = (missing_data / len(df_tmp)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing %': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])

# 3. Kiểm tra dữ liệu sau khi lọc
print("\n3. DỮ LIỆU SAU KHI LỌC:")
print(f"   - Giá min: {df_tmp['gia(ty)'].min():.3f} tỷ")
print(f"   - Giá max: {df_tmp['gia(ty)'].max():.3f} tỷ")
print(f"   - Giá trung bình: {df_tmp['gia(ty)'].mean():.3f} tỷ")
print(f"   - Diện tích min: {df_tmp['dien_tich_su_dung_m2'].min():.1f} m²")
print(f"   - Diện tích max: {df_tmp['dien_tich_su_dung_m2'].max():.1f} m²")
print(f"   - Diện tích trung bình: {df_tmp['dien_tich_su_dung_m2'].mean():.1f} m²")

# 4. Kiểm tra giá/m² sau khi lọc
df_tmp['gia_per_m2'] = (df_tmp['gia(ty)'] * 1000) / df_tmp['dien_tich_su_dung_m2']
print(f"   - Giá/m² min: {df_tmp['gia_per_m2'].min():.0f} triệu VND/m²")
print(f"   - Giá/m² max: {df_tmp['gia_per_m2'].max():.0f} triệu VND/m²")
print(f"   - Giá/m² trung bình: {df_tmp['gia_per_m2'].mean():.0f} triệu VND/m²")

# 5. Kiểm tra phân phối theo quận
print("\n4. PHÂN PHỐI THEO QUẬN:")
quan_distribution = df_tmp['quan_huyen'].value_counts().head(10)
print(quan_distribution)

# 6. Kiểm tra phân phối theo pháp lý
print("\n5. PHÂN PHỐI THEO PHÁP LÝ:")
phap_ly_distribution = df_tmp['phap_ly'].value_counts()
print(phap_ly_distribution)

# 7. Kiểm tra dữ liệu theo thời gian
print("\n6. PHÂN PHỐI THEO THỜI GIAN:")
df_tmp['year'] = df_tmp['ngay_dang'].dt.year
year_distribution = df_tmp['year'].value_counts().sort_index()
print(year_distribution)


In [15]:
df_tmp.to_csv("batdongsan_clean.csv", index=False, encoding="utf-8-sig")


In [14]:
df_tmp[df_tmp['gia(ty)'] == 0.0]

,ma_bds,dien_tich_su_dung_m2,gia(ty),phap_ly,ngay_dang,phuong_xa,quan_huyen,thanh_pho,nha_tam,phong_ngu,dien_tich_dat_m2,gia_per_m2


In [17]:

# Thay thế NaN trong dien_tich_su_dung_m2 bằng dien_tich_dat_m2
mask_su_dung_na = df_tmp['dien_tich_su_dung_m2'].isna() & df_tmp['dien_tich_dat_m2'].notna()
df_tmp.loc[mask_su_dung_na, 'dien_tich_su_dung_m2'] = df_tmp.loc[mask_su_dung_na, 'dien_tich_dat_m2']

# Thay thế NaN trong dien_tich_dat_m2 bằng dien_tich_su_dung_m2
mask_dat_na = df_tmp['dien_tich_dat_m2'].isna() & df_tmp['dien_tich_su_dung_m2'].notna()
df_tmp.loc[mask_dat_na, 'dien_tich_dat_m2'] = df_tmp.loc[mask_dat_na, 'dien_tich_su_dung_m2']



In [19]:
df_tmp.head(5)
df_tmp.to_csv("batdongsan_clean.csv", index=False, encoding="utf-8-sig")



In [ ]:
# ===== UPLOAD CSV LÊN GOOGLE CLOUD STORAGE =====

from google.cloud import storage
import os
from datetime import datetime

# Cấu hình GCP
PROJECT_ID = "your-project-id"  # Thay bằng Project ID của bạn
BUCKET_NAME = "your-bucket-name"  # Thay bằng tên bucket của bạn
BLOB_NAME = f"batdongsan_clean_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

def upload_csv_to_gcs(file_path, bucket_name, blob_name, project_id=None):
    """
    Upload file CSV lên Google Cloud Storage
    
    Args:
        file_path (str): Đường dẫn đến file CSV local
        bucket_name (str): Tên bucket trên GCS
        blob_name (str): Tên file sẽ lưu trên GCS
        project_id (str): Project ID của GCP
    
    Returns:
        bool: True nếu upload thành công, False nếu thất bại
    """
    try:
        # Khởi tạo client
        if project_id:
            client = storage.Client(project=project_id)
        else:
            client = storage.Client()
        
        # Lấy bucket
        bucket = client.bucket(bucket_name)
        
        # Tạo blob object
        blob = bucket.blob(blob_name)
        
        # Upload file
        blob.upload_from_filename(file_path)
        
        print(f"✅ Upload thành công!")
        print(f"📁 File: {file_path}")
        print(f"🪣 Bucket: {bucket_name}")
        print(f"📄 Blob name: {blob_name}")
        print(f"🔗 URL: gs://{bucket_name}/{blob_name}")
        
        return True
        
    except Exception as e:
        print(f"❌ Lỗi khi upload: {str(e)}")
        return False

# Kiểm tra file CSV clean có tồn tại không
csv_file_path = "batdongsan_clean.csv"
if os.path.exists(csv_file_path):
    print(f"📋 File CSV clean đã được tìm thấy: {csv_file_path}")
    print(f"📊 Kích thước file: {os.path.getsize(csv_file_path)} bytes")
else:
    print("❌ Không tìm thấy file batdongsan_clean.csv")
    print("💡 Hãy chạy lại cell trước để tạo file clean")


In [ ]:
# ===== THIẾT LẬP AUTHENTICATION CHO GCP =====

# Cách 1: Sử dụng Service Account Key File (Khuyến nghị cho development)
# Uncomment dòng dưới và thay đổi đường dẫn đến file key của bạn
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/service-account-key.json'

# Cách 2: Sử dụng Application Default Credentials (ADC)
# Chạy lệnh: gcloud auth application-default login
# Hoặc set environment variable: GOOGLE_APPLICATION_CREDENTIALS

# Kiểm tra authentication
try:
    from google.cloud import storage
    client = storage.Client()
    # Test kết nối bằng cách list buckets
    buckets = list(client.list_buckets())
    print("✅ Authentication thành công!")
    print(f"🔍 Tìm thấy {len(buckets)} bucket(s) trong project")
    for bucket in buckets[:3]:  # Hiển thị 3 bucket đầu tiên
        print(f"   📦 {bucket.name}")
    if len(buckets) > 3:
        print(f"   ... và {len(buckets) - 3} bucket khác")
except Exception as e:
    print("❌ Lỗi authentication:")
    print(f"   {str(e)}")
    print("\n💡 Hướng dẫn setup:")
    print("   1. Tạo Service Account Key từ GCP Console")
    print("   2. Download file JSON key")
    print("   3. Set environment variable: GOOGLE_APPLICATION_CREDENTIALS")
    print("   4. Hoặc chạy: gcloud auth application-default login")


In [ ]:
# ===== THỰC HIỆN UPLOAD CSV LÊN GCP =====

# Cấu hình thông tin GCP của bạn
PROJECT_ID = "your-project-id"  # Thay bằng Project ID thực của bạn
BUCKET_NAME = "your-bucket-name"  # Thay bằng tên bucket thực của bạn

# Tạo tên file với timestamp
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
BLOB_NAME = f"batdongsan_clean_{timestamp}.csv"

print("🚀 Bắt đầu upload file CSV lên Google Cloud Storage...")
print(f"📁 File source: {csv_file_path}")
print(f"🪣 Destination bucket: {BUCKET_NAME}")
print(f"📄 Destination filename: {BLOB_NAME}")
print("-" * 50)

# Thực hiện upload
success = upload_csv_to_gcs(
    file_path=csv_file_path,
    bucket_name=BUCKET_NAME,
    blob_name=BLOB_NAME,
    project_id=PROJECT_ID
)

if success:
    print("\n🎉 Upload hoàn tất!")
    print(f"📋 File đã được lưu tại: gs://{BUCKET_NAME}/{BLOB_NAME}")
    print("\n💡 Bạn có thể:")
    print("   1. Xem file trên GCP Console")
    print("   2. Sử dụng file này cho BigQuery, Dataflow, hoặc các dịch vụ khác")
    print("   3. Chia sẻ link với team members")
else:
    print("\n❌ Upload thất bại!")
    print("💡 Vui lòng kiểm tra:")
    print("   - Authentication đã được thiết lập đúng chưa")
    print("   - Bucket name có tồn tại không")
    print("   - Bạn có quyền write vào bucket không")
    print("   - File CSV có tồn tại không")


In [ ]:
# ===== VERIFY UPLOAD & THÔNG TIN BỔ SUNG =====

def verify_upload(bucket_name, blob_name):
    """Kiểm tra file đã được upload thành công"""
    try:
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        
        if blob.exists():
            # Lấy thông tin file
            blob.reload()
            size = blob.size
            created = blob.time_created
            updated = blob.updated
            
            print("✅ File đã tồn tại trên GCS!")
            print(f"📊 Kích thước: {size:,} bytes ({size/1024/1024:.2f} MB)")
            print(f"📅 Tạo lúc: {created}")
            print(f"🔄 Cập nhật lúc: {updated}")
            print(f"🔗 URL: gs://{bucket_name}/{blob_name}")
            return True
        else:
            print("❌ File chưa tồn tại trên GCS")
            return False
            
    except Exception as e:
        print(f"❌ Lỗi khi verify: {str(e)}")
        return False

# Verify upload nếu đã thành công
if 'success' in locals() and success:
    print("\n🔍 Đang verify file upload...")
    verify_upload(BUCKET_NAME, BLOB_NAME)

# Thông tin bổ sung về file CSV
print(f"\n📋 THÔNG TIN FILE CSV:")
print(f"   📁 File: {csv_file_path}")
print(f"   📊 Số dòng: {len(df_tmp):,}")
print(f"   📊 Số cột: {len(df_tmp.columns)}")
print(f"   📊 Các cột: {list(df_tmp.columns)}")
print(f"   💾 Kích thước: {os.path.getsize(csv_file_path):,} bytes")


# 🚀 HƯỚNG DẪN SETUP GCP ĐỂ UPLOAD CSV

## 📋 Các bước cần thiết:

### 1. 🔑 Authentication Setup
**Cách 1: Service Account Key (Khuyến nghị)**
```bash
# 1. Tạo Service Account từ GCP Console
# 2. Download file JSON key
# 3. Set environment variable:
export GOOGLE_APPLICATION_CREDENTIALS="path/to/your/service-account-key.json"
```

**Cách 2: Application Default Credentials**
```bash
# Cài đặt gcloud CLI và chạy:
gcloud auth application-default login
```

### 2. 🪣 Tạo Bucket (nếu chưa có)
```bash
# Tạo bucket mới
gsutil mb gs://your-bucket-name

# Hoặc từ GCP Console: Storage > Buckets > Create bucket
```

### 3. ⚙️ Cấu hình trong code
Thay đổi các biến trong cell upload:
- `PROJECT_ID`: Project ID của bạn trên GCP
- `BUCKET_NAME`: Tên bucket đã tạo

### 4. 📦 Cài đặt dependencies
```bash
pip install google-cloud-storage
```

## 🔍 Kiểm tra quyền truy cập:
- Service Account cần có role: `Storage Object Admin` hoặc `Storage Admin`
- Bucket phải tồn tại và có quyền write

## 📊 Sau khi upload thành công:
- File sẽ có URL: `gs://bucket-name/filename.csv`
- Có thể sử dụng cho BigQuery, Dataflow, ML pipelines
- Có thể chia sẻ với team members


In [ ]:
# ===== FIX: ĐỊNH NGHĨA LẠI FUNCTION UPLOAD =====

from google.cloud import storage
import os
from datetime import datetime

def upload_csv_to_gcs(file_path, bucket_name, blob_name, project_id=None):
    """
    Upload file CSV lên Google Cloud Storage
    
    Args:
        file_path (str): Đường dẫn đến file CSV local
        bucket_name (str): Tên bucket trên GCS
        blob_name (str): Tên file sẽ lưu trên GCS
        project_id (str): Project ID của GCP
    
    Returns:
        bool: True nếu upload thành công, False nếu thất bại
    """
    try:
        # Khởi tạo client
        if project_id:
            client = storage.Client(project=project_id)
        else:
            client = storage.Client()
        
        # Lấy bucket
        bucket = client.bucket(bucket_name)
        
        # Tạo blob object
        blob = bucket.blob(blob_name)
        
        # Upload file
        blob.upload_from_filename(file_path)
        
        print(f"✅ Upload thành công!")
        print(f"📁 File: {file_path}")
        print(f"🪣 Bucket: {bucket_name}")
        print(f"📄 Blob name: {blob_name}")
        print(f"🔗 URL: gs://{bucket_name}/{blob_name}")
        
        return True
        
    except Exception as e:
        print(f"❌ Lỗi khi upload: {str(e)}")
        return False

print("✅ Function upload_csv_to_gcs đã được định nghĩa!")


In [ ]:
bucket_name =  'hanoi-bds-raw-data'
source_file_name = 'batdongsan_clean.csv'
destination_file_name = 'batdongsan_clean.csv'
upload_csv_to_gcs(source_file_name, bucket_name, destination_file_name)

NameError: name 'upload_csv_to_gcs' is not defined